# Feature generation

## Feature generated

In this notebook we will generate the following features bases on the dataset : 

imbalance of the book-order : 

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../data/DATA_0/ETH_EUR.csv')
